# Table of Contents<a name="toc"></a>
* [Convert hex to base64](#prob1)
* [Fixed XOR](#prob2)
* [Single-byte XOR cipher](#prob3)

# Convert hex to base64<a name="prob1"></a>

In [1]:
string_to_convert = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

In [2]:
import binascii, base64
output = base64.b64encode(binascii.unhexlify(string_to_convert)).decode()
print(f"base64 encoded: {output}")

base64 encoded: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


# Fixed XOR<a name="prob2"></a>

In [3]:
%%capture
!pip install numpy
import numpy as np

In [4]:
cipher = "1c0111001f010100061a024b53535009181c"

In [5]:
xor_block = "686974207468652062756c6c277320657965"

In [6]:
def xor_encrypt(cipher: bytes, block: bytes) -> bytes:
    cipher_npa = np.frombuffer(cipher, dtype=np.uint8)
    block_npa = np.frombuffer(block, dtype=np.uint8)
    return np.bitwise_xor(cipher_npa, block_npa).tobytes()

In [7]:
ciphertext = xor_encrypt(binascii.unhexlify(cipher), binascii.unhexlify(xor_block))
print(f"ciphertext, hex: {binascii.hexlify(ciphertext).decode()}")
print(f"ciphertext, bytes: {ciphertext}")

ciphertext, hex: 746865206b696420646f6e277420706c6179
ciphertext, bytes: b"the kid don't play"


# Single-byte XOR cipher<a name="prob3"></a>

In [8]:
ciphertext = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

Iterate over all possible XOR values from 0x00 to 0xFF, and then compare all characters in the candidate cipher output against valid printable ASCII values, which range from 0x20 - 0x7E.

**Assumption** is that the output is printable ASCII.